## TF-IDF를 이용해 유사한 제목 추천
* 먼저, 유튜브 데이터를 크롤링(후에 Youtube Data api를 이용해 수정)
* TF-IDF와 코사인 유사도를 이용해 유사한 제목 출력<br>
[유튜브 크롤링](https://heytech.tistory.com/325)<br>
[TF-IDF 를 활용한 제목별 추천 시스템](https://acdongpgm.tistory.com/175)

In [21]:
# 유튜브 데이터 크롤링
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

In [22]:
# 최신 크롬 드라이버 사용하도록 세팅: 현재 OS에 설치된 크롬 브라우저 버전에 맞게 cache에 드라이버 설치
from selenium.webdriver.chrome.service import Service
service = Service(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\ing06\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [23]:
# 크롤링 함수
def scroll():
    try:        
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            pause_time = random.uniform(1, 2)
            # 페이지 최하단까지 스크롤
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            time.sleep(pause_time)
            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if new_page_height == last_page_height:
                print("스크롤 완료")
                break
                
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

In [25]:
# 검색 키워드 설정: 키워드 내 띄어쓰기는 URL에서 '+'로 표시되기 때문에 이에 맞게 변환
SEARCH_KEYWORD = '동물'.replace(' ', '+')

# 드라이버 세팅 및 실행
driver = webdriver.Chrome(service=service)
# 스크래핑 할 URL 세팅
URL = "https://www.youtube.com/results?search_query=" + SEARCH_KEYWORD
# 크롬 드라이버를 통해 지정한 URL의 웹 페이지 오픈
driver.get(URL)
# 웹 페이지 로딩 대기
time.sleep(3)
# 무한 스크롤 함수 실행
scroll()

스크롤 완료


In [26]:
# 페이지 소스 추출
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

In [27]:
# 데이터 추출
# 모든 콘텐츠 정보
contents = soup_source.find_all(class_ = 'yt-simple-endpoint style-scope ytd-video-renderer')
# 콘텐츠 제목만 추출(beautifulsoup의 get_text() 이용)
contents_title = list(map(lambda data: data.get_text().replace("\n", ""), contents))
# 콘텐츠 링크만 추출
contents_link = list(map(lambda data: "https://youtube.com" + data["href"], contents))
# 제목, 링크를 딕셔너리 형태로
contents_dict = {'title' : contents_title, 'link': contents_link}

In [28]:
# 데이터프레임으로
df_contents = pd.DataFrame(contents_dict)
df_contents

,title,link
0,"상어, 기린, 호랑이, 고릴라, 낙타는 영어로?ㅣ영어 배우기ㅣ위키와 동물언어_영어ㅣ...",https://youtube.com/watch?v=HRd98hTZb-U
1,절대로 부활하면 안 되는 멸종 동물!,https://youtube.com/watch?v=c20uGI5Mmvs
2,심쿵 아기동물 성장기 #OfftheFence #KBS #동물의왕국 (KBS1 202...,https://youtube.com/watch?v=p_wTEHPGAGI
3,본격 귀여움 참기 챌린지! 역대급 심장 폭행범 ‘꼬물이들.zip’ I TV동물농장 ...,https://youtube.com/watch?v=c9cYIGqdcvA
4,귀엽고 신기한 동물들을 만나봤어요! 동물편 모음 40분 자연 학습 체험,https://youtube.com/watch?v=VotU3AUcMuk
...,...,...
774,To Rombi Episode 45: Zombie combine with S.W.A...,https://youtube.com/watch?v=-pT0DbboyMc
775,It is believed that cats are afraid of water ...,https://youtube.com/watch?v=ENg56WXqjRk
776,【대식】모츠 조림 정식의 라이스×10으로 해 본 결과…【몬 요시】,https://youtube.com/watch?v=4NKx6qF_yMc
777,KRISIS DUA BINATANG ROH TERKUAT DA MING DAN ER...,https://youtube.com/watch?v=j7QVk1vxnWM


In [29]:
# 데이터 저장
df_contents.to_csv("animals_crawling.csv", encoding='utf-8-sig')

In [30]:
# tf-idf로 유사 제목 추천
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
tfidf = TfidfVectorizer(stop_words='english')  # 불용어는 일단 영어로 지정..
tfidf_matrix = tfidf.fit_transform(df_contents['title'])
print(tfidf_matrix.shape)  # (743, 3691), 3691은 743개의 제목에 있는 단어들을 분리한 개수

(779, 3974)


In [32]:
# 코사인 유사도로 행렬과 행렬 간의 유사도 비교
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_matrix.shape
print(cosine_matrix)

[[1.         0.         0.03985572 ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.03985572 0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [33]:
# title과 id를 매핑할 dictinary
title2idx = {}
for i, c in enumerate(df_contents['title']): 
    title2idx[i] = c

# id와 title를 매핑할 dictionary
idx2title = {}
for i, c in title2idx.items(): 
    idx2title[c] = i

In [35]:
# title을 넣어 해당 항목과 유사한 상위 10개 추출
idx = idx2title['절대로 부활하면 안 되는 멸종 동물!']
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:10] # 상위 10개의 인덱스와 유사도를 추출 

[(214, 0.24004916281888936),
 (63, 0.1561699465538434),
 (238, 0.14311138130518267),
 (80, 0.08564214439759417),
 (505, 0.0808179334528569),
 (577, 0.08030556502487728),
 (148, 0.08015430924465278),
 (225, 0.07974748587196744),
 (544, 0.07831085678674914),
 (582, 0.07027682671185677)]

In [36]:
# 인덱스를 다시 제목으로
sim_scores = [(title2idx[i], score) for i, score in sim_scores[0:10]]
sim_scores

[('한 평생 구애만 하다 죽게 되는 동물', 0.24004916281888936),
 ('🌏동물 친구들과 지구를 보호해요! | 🦁멸종 위기 동물, 동물활동가, 분리수거송 | Save the Earth | 환경보호 | 동물동요 | 환경동요 | 주니토니 | JunyTony',
  0.1561699465538434),
 ('멸종 위기 동물을 보호해요 | 🌏지구가 아프대! 🐼동물 친구들이 위험해! | 동물동요 | 초록초록 지구 | 환경동요 | 주니토니 | JunyTony',
  0.14311138130518267),
 ('한국어ㅣ위기에 빠진 동물 구출하기 6! 어린이 동물 만화,  동물 이름 외우기ㅣ꼬꼬스토이', 0.08564214439759417),
 ('진흙에 빠진 숲속 동물 구하기ㅣ어린이 동물 동화,  동물 만화ㅣ꼬꼬스토이', 0.0808179334528569),
 ('말썽꾸러기 곰과 숲속 동물 친구들ㅣ어린이 동물 만화, 동물 동화ㅣ꼬꼬스토이', 0.08030556502487728),
 ('위기에 빠진 동물 구출하기 7! 어린이 동물 만화,  동물 이름 외우기, 컬렉타피규어ㅣ꼬꼬스토이',
  0.08015430924465278),
 ('한국어ㅣ아빠에게 선물받은 동물 장난감 컬렉션!, 어린이 동물 만화, 동물 이름 외우기ㅣ꼬꼬스토이',
  0.07974748587196744),
 ('동물 수송 트럭 농장 동물!', 0.07831085678674914),
 ('🐙문어 의사 선생님 | 동물 동요 | 바닷속 동물 병원 | 똑똑한 바다 동물 | 꼬마공룡 코코비',
  0.07027682671185677)]